In [ ]:
!nvidia-smi

In [2]:
import torch
import torchvision
from PIL import Image
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torchvision.transforms.functional as F
import os

In [3]:
print(torch.cuda.get_device_properties(0))

device = "cuda" if torch.cuda.is_available() else "cpu"

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


# **LEVEL1- Use pretrained FasterRCNN model**

*Open & display sample image*

In [ ]:
img_pil = Image.open("/content/obj-det-test.jpg").convert("RGB")
display(img_pil)

*Define pretrained FasterRcnn pytorch model*

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)
model.to(device)

*Evaluation model*

In [ ]:
img_tensor = F.to_tensor(img_pil)
img_list   = [img_tensor.to(device)]

model.eval()
with torch.no_grad():
    prediction = model(img_list)

prediction

In [ ]:
boxes  = prediction[0]["boxes"]
labels = prediction[0]["labels"]
scores = prediction[0]["scores"]

In [ ]:
from matplotlib.patches import Rectangle

img_tensor_np = img_tensor.permute(1, 2, 0).cpu().numpy()
plt.figure(figsize = (15, 15))
plt.imshow(img_tensor_np)

ax = plt.gca()
for box, label, score in zip(boxes, labels, scores):
    box = box.cpu().numpy()
    score = score.cpu().numpy()
    if score > 0.6:
        rect = Rectangle((box[0],  box[1]),
                        (box[2] - box[0]),
                        (box[3] - box[1]),
                        fill = False, edgecolor = (1, 0, 0), linewidth = 2)
        ax.add_patch(rect)

plt.show()

*Train* pytorch pretrain FasterRCNN 

In [ ]:
targets = []
target  = {}
target["boxes"]  = boxes
target["labels"] = labels
targets.append(target)

model.train()
loss = model(img_list, targets)
loss

# **LEVEL2- Use pytorch FasterRCNN model in raccoon dataset**

In [4]:
!git clone https://github.com/experiencor/raccoon_dataset

Cloning into 'raccoon_dataset'...
remote: Enumerating objects: 646, done.
remote: Total 646 (delta 0), reused 0 (delta 0), pack-reused 646
Receiving objects: 100% (646/646), 48.00 MiB | 17.71 MiB/s, done.
Resolving deltas: 100% (412/412), done.


In [ ]:
root  = "/content/raccoon_dataset"
phase = "train"

images  = os.listdir(os.path.join(root, "images"))
targets = pd.read_csv(os.path.join(root, "data/{}_labels.csv".format(phase)))

idx = "raccoon-119.jpg"
img_path = os.path.join(root, "images/{}".format(idx))
img = Image.open(img_path).convert("RGB")
display(img)

box_list = targets[targets["filename"] == "raccoon-119.jpg"]
box_list = box_list[["xmin", "ymin", "xmax", "ymax"]].values
print(box_list)

*plot one random sample from dataset*

In [ ]:
from matplotlib.patches import Rectangle

img_tensor = F.to_tensor(img).permute(1, 2, 0).cpu().numpy()

plt.figure(figsize = (12,12))
plt.imshow(img_tensor)

ax = plt.gca()
for l in box_list:
    rect = Rectangle((l[0],  l[1]),
                     (l[2] - l[0]),
                     (l[3] - l[1]),
                     fill = False, edgecolor = (1, 0, 0), linewidth = 2)
    ax.add_patch(rect)

plt.show()


## **Costum dataset**

In [ ]:
class RaccoonDataset(torch.utils.data.Dataset):

    def __init__(self, root, phase):
        self.root  = root
        self.phase = phase

        #self.images_name = os.listdir(os.path.join(self.root, "images"))
        self.targets     = pd.read_csv(os.path.join(self.root, "data/{}_labels.csv".format(self.phase)))
        self.images_name = self.targets["filename"]

    def __getitem__(self, idx):
        # images
        img_path   = os.path.join(self.root, "images", self.images_name[idx])
        img        = Image.open(img_path).convert("RGB")
        img_tensor = F.to_tensor(img)

        # boxes
        target     = self.targets[self.targets["filename"] == self.images_name[idx]]
        boxes_list = target[["xmin", "ymin", "xmax", "ymax"]].values   # boxes is a numpy array
        boxes      = torch.tensor(boxes_list, dtype = torch.float32)

        # labels
        labels     = torch.ones((len(boxes_list), ), dtype = torch.int64)

        #rac_targets = []
        rac_target  = {}
        rac_target["boxes"]  = boxes
        rac_target["labels"] = labels
        #rac_targets.append(rac_target)

        return img_tensor, rac_target

    def __len__(self):
        return len(self.images_name)


In [35]:
trainset = RaccoonDataset(root = "/content/raccoon_dataset", phase = "train")
testset  = RaccoonDataset(root = "/content/raccoon_dataset", phase = "test")

In [36]:
trainset.__getitem__(10)

(tensor([[[0.7294, 0.7294, 0.7255,  ..., 0.4431, 0.4353, 0.4235],
          [0.7294, 0.7294, 0.7255,  ..., 0.4353, 0.4314, 0.4431],
          [0.7294, 0.7294, 0.7255,  ..., 0.4863, 0.4824, 0.4784],
          ...,
          [0.2980, 0.3333, 0.4314,  ..., 0.4353, 0.4588, 0.4588],
          [0.3490, 0.3412, 0.3294,  ..., 0.4118, 0.4275, 0.4235],
          [0.3569, 0.3412, 0.3686,  ..., 0.4157, 0.4275, 0.4275]],
 
         [[0.6314, 0.6314, 0.6314,  ..., 0.3529, 0.3412, 0.3294],
          [0.6314, 0.6314, 0.6314,  ..., 0.3412, 0.3373, 0.3490],
          [0.6314, 0.6314, 0.6314,  ..., 0.3922, 0.3882, 0.3843],
          ...,
          [0.2706, 0.3059, 0.4039,  ..., 0.3098, 0.3333, 0.3333],
          [0.3216, 0.3137, 0.3020,  ..., 0.2863, 0.3020, 0.2980],
          [0.3216, 0.3059, 0.3333,  ..., 0.2902, 0.3020, 0.3020]],
 
         [[0.4745, 0.4745, 0.4588,  ..., 0.2980, 0.2941, 0.2824],
          [0.4745, 0.4706, 0.4588,  ..., 0.2941, 0.2902, 0.3020],
          [0.4706, 0.4706, 0.4588,  ...,

In [37]:
def new_concat(batch):
    return tuple(zip(*batch))

In [38]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 2, 
                                          shuffle = True, num_workers = 2, collate_fn = new_concat)
testloader  = torch.utils.data.DataLoader(testset,  batch_size = 1,
                                          shuffle = True, num_workers = 2, collate_fn = new_concat)

In [39]:
batch = next(iter(trainloader))
batch

((tensor([[[0.4588, 0.4745, 0.4902,  ..., 0.5961, 0.6157, 0.6314],
           [0.4745, 0.4824, 0.4863,  ..., 0.5333, 0.5647, 0.6000],
           [0.4863, 0.4863, 0.4824,  ..., 0.5137, 0.5412, 0.5765],
           ...,
           [0.2667, 0.2745, 0.2863,  ..., 0.3137, 0.3176, 0.3529],
           [0.2784, 0.2863, 0.2941,  ..., 0.3255, 0.3294, 0.3373],
           [0.2784, 0.2863, 0.2941,  ..., 0.3255, 0.3294, 0.3373]],
  
          [[0.6588, 0.6745, 0.6824,  ..., 0.6863, 0.7020, 0.7176],
           [0.6667, 0.6745, 0.6784,  ..., 0.6275, 0.6549, 0.6902],
           [0.6824, 0.6784, 0.6745,  ..., 0.6039, 0.6314, 0.6667],
           ...,
           [0.3490, 0.3569, 0.3686,  ..., 0.1804, 0.1725, 0.2078],
           [0.3490, 0.3569, 0.3647,  ..., 0.1922, 0.1961, 0.2039],
           [0.3490, 0.3569, 0.3647,  ..., 0.1922, 0.1961, 0.2039]],
  
          [[0.1529, 0.1686, 0.1804,  ..., 0.5216, 0.5490, 0.5686],
           [0.1647, 0.1725, 0.1765,  ..., 0.4392, 0.4745, 0.5176],
           [0.1686, 0.

## Define model

In [40]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(1024, 2)
model.to(device)
model

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

## model config

In [41]:
optimizer = torch.optim.Adam(params = model.parameters(), lr = 0.0002, weight_decay = 1e-5)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer = optimizer, step_size = 3, gamma = 0.5)

In [43]:
def train_one_epoch(model, optimizer, train_loader):
    model.train()
    loss_total = 0
    for images, targets in train_loader:
        optimizer.zero_grad()
        images      = [img.to(device) for img in images]
        targets     = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict   = model(images, targets)
        loss        = sum(loss for loss in loss_dict.values())
        loss_total += loss
        loss.backward()
        optimizer.step()


    return loss_total / len(train_loader)


In [ ]:
!pip install termcolor2
from termcolor2 import colored 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for termcolor2: filename=termcolor2-0.0.3-py3-none-any.whl size=1833 sha256=905ef6e5f9dd55711ca6b7b5496e8c8a57863f68d6170cccc053d66c0c695f01
  Stored in directory: /root/.cache/pip/wheels/37/93/fa/3ead517c143d4381f9b4096524f21b1d86a5fa60d08210abe8
Successfully built termcolor2


In [44]:
import datetime
train_epochs = 50

for epoch in range(train_epochs):
    s_time = datetime.datetime.now()
    loss   = train_one_epoch(model, optimizer, trainloader)
    e_time = datetime.datetime.now()
    train_time = str(e_time - s_time)
    print(colored("Epoch: [{}] \t ".format(epoch + 1)),
          colored(f"Time: {train_time} \t ", color = "blue"),
          colored("Loss: {0:.4} \t ".format(loss), color = "red"),
          colored("LR: {}  ".format(lr_scheduler.get_last_lr()), color = "green")) 


    lr_scheduler.step()

Epoch: [1] 	  Time: 0:00:53.622446 	  Loss: 0.2422 	  LR: [0.0002]  
Epoch: [2] 	  Time: 0:00:51.092353 	  Loss: 0.2344 	  LR: [0.0002]  
Epoch: [3] 	  Time: 0:00:50.798165 	  Loss: 0.2385 	  LR: [0.0002]  
Epoch: [4] 	  Time: 0:00:51.544479 	  Loss: 0.2094 	  LR: [0.0001]  
Epoch: [5] 	  Time: 0:00:52.077975 	  Loss: 0.1864 	  LR: [0.0001]  
Epoch: [6] 	  Time: 0:00:51.557349 	  Loss: 0.1744 	  LR: [0.0001]  
Epoch: [7] 	  Time: 0:00:51.622532 	  Loss: 0.1563 	  LR: [5e-05]  
Epoch: [8] 	  Time: 0:00:51.186432 	  Loss: 0.1444 	  LR: [5e-05]  
Epoch: [9] 	  Time: 0:00:51.496776 	  Loss: 0.1315 	  LR: [5e-05]  
Epoch: [10] 	  Time: 0:00:51.521102 	  Loss: 0.1201 	  LR: [2.5e-05]  
Epoch: [11] 	  Time: 0:00:51.153247 	  Loss: 0.119 	  LR: [2.5e-05]  
Epoch: [12] 	  Time: 0:00:51.186889 	  Loss: 0.108 	  LR: [2.5e-05]  
Epoch: [13] 	  Time: 0:00:51.914190 	  Loss: 0.1027 	  LR: [1.25e-05]  
Epoch: [14] 	  Time: 0:00:52.143835 	  Loss: 0.09961 	  LR: [1.25e-05]  
Epoch: [15] 	  Time: 0:00:

In [45]:
!mkdir /content/output_images

In [46]:
from matplotlib.patches import Rectangle


def evaluate(model, test_loader):
    model.eval()
    with torch.no_grad():
        cnt = 0
        for images, targets in test_loader:
            images  = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            out     = model(images)
            scores  = out[0]["scores"].cpu().numpy()
            inds    = scores > 0.7
            boxes   = out[0]["boxes"].cpu().numpy()
            bxs     = boxes[inds]

            # ground Truth & image
            gt      = targets[0]["boxes"].cpu().numpy()
            image   = images[0].permute(1, 2, 0).cpu().numpy()
            #---------------------------------------------
            fig, ax = plt.subplots(1)
            ax.imshow(image)
            for j in range(len(gt)):
                rect1 = Rectangle((int(gt[j][0]), int(gt[j][1])), 
                                   abs(gt[j][2] - gt[j][0]),
                                   abs(gt[j][3] - gt[j][1]),
                                   fill = False, edgecolor = (0, 1, 0), linewidth = 3)
                ax.add_patch(rect1)

            for i in range(len(bxs)):
                rect2 = Rectangle((int(bxs[i][0]), int(bxs[i][1])),
                                   abs(bxs[i][2] - bxs[i][0]),
                                   abs(bxs[i][3] - bxs[i][1]),
                                   fill = False, edgecolor = (1, 0, 0), linewidth = 3)
                ax.add_patch(rect2)

            fig.savefig("/content/output_images/{}.png".format(cnt), dpi = 90, bbox_inches = "tight")
            cnt += 1




**plot image and box in one fig**

In [ ]:
from matplotlib.patches import Rectangle


def evaluate(model, test_loader):
    model.eval()
    with torch.no_grad():
        cnt = 0
        fig = plt.figure(figsize = (20, 20))
        for batch_idx ,(images, targets) in enumerate(test_loader):
            images  = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            out     = model(images)
            scores  = out[0]["scores"].cpu().numpy()
            inds    = scores > 0.7
            boxes   = out[0]["boxes"].cpu().numpy()
            bxs     = boxes[inds]

            # ground Truth & image
            gt      = targets[0]["boxes"].cpu().numpy()
            image   = images[0].permute(1, 2, 0).cpu().numpy()
            #---------------------------------------------
            n_images = len(test_loader)
            ax  = fig.add_subplot(n_images/ 4 , 11 , batch_idx + 1 )
            ax.imshow(image)
            for j in range(len(gt)):
                rect1 = Rectangle((int(gt[j][0]), int(gt[j][1])), 
                                   abs(gt[j][2] - gt[j][0]),
                                   abs(gt[j][3] - gt[j][1]),
                                   fill = False, edgecolor = (0, 1, 0), linewidth = 3)
                ax.add_patch(rect1)

            for i in range(len(bxs)):
                rect2 = Rectangle((int(bxs[i][0]), int(bxs[i][1])),
                                   abs(bxs[i][2] - bxs[i][0]),
                                   abs(bxs[i][3] - bxs[i][1]),
                                   fill = False, edgecolor = (1, 0, 0), linewidth = 3)
                ax.add_patch(rect2)

            fig.savefig("/content/output_images/{}.png".format(cnt), dpi = 90, bbox_inches = "tight")
            cnt += 1




In [ ]:
evaluate(model, testloader)

# **Level3- Use mobilenet backbone in FasterRCNN model**

In [ ]:
backbone    = torchvision.models.mobilenet_v2(pretrained = True).features
backbone.out_channels = 1280
anchor_gen = torchvision.models.detection.rpn.AnchorGenerator(sizes = ((32, 64, 128, 256, 512), ),
                                                      aspect_ratios = ((0.5, 1.0, 2.0), ))

light_model = torchvision.models.detection.FasterRCNN(backbone = backbone, num_classes = 2,
                                                       rpn_anchor_generator = anchor_gen)
light_model.to(device)
light_model

In [6]:
class RaccoonDataset(torch.utils.data.Dataset):

    def __init__(self, root, phase):
        self.root  = root
        self.phase = phase

        #self.images_name = os.listdir(os.path.join(self.root, "images"))
        self.targets     = pd.read_csv(os.path.join(self.root, "data/{}_labels.csv".format(self.phase)))
        self.images_name = self.targets["filename"]

    def __getitem__(self, idx):
        # images
        img_path   = os.path.join(self.root, "images", self.images_name[idx])
        img        = Image.open(img_path).convert("RGB")
        img_tensor = F.to_tensor(img)

        # boxes
        target     = self.targets[self.targets["filename"] == self.images_name[idx]]
        boxes_list = target[["xmin", "ymin", "xmax", "ymax"]].values   # boxes is a numpy array
        boxes      = torch.tensor(boxes_list, dtype = torch.float32)

        # labels
        labels     = torch.ones((len(boxes_list), ), dtype = torch.int64)

        #rac_targets = []
        rac_target  = {}
        rac_target["boxes"]  = boxes
        rac_target["labels"] = labels
        #rac_targets.append(rac_target)

        return img_tensor, rac_target

    def __len__(self):
        return len(self.images_name)


In [21]:
trainset = RaccoonDataset(root = "/content/raccoon_dataset", phase = "train")
testset  = RaccoonDataset(root = "/content/raccoon_dataset", phase = "test")

In [22]:
def new_concat(batch):
    return tuple(zip(*batch))

In [23]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 2, 
                                          shuffle = True, num_workers = 2, collate_fn = new_concat)
testloader  = torch.utils.data.DataLoader(testset,  batch_size = 1,
                                          shuffle = True, num_workers = 2, collate_fn = new_concat)

In [24]:
optimizer = torch.optim.Adam(params = light_model.parameters(), lr = 0.0002, weight_decay = 5e-4)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer = optimizer, step_size = 10, gamma = 0.5)

In [25]:
def train_one_epoch(model, optimizer, train_loader):
    model.train()
    loss_total = 0
    for images, targets in train_loader:
        optimizer.zero_grad()
        images      = [img.to(device) for img in images]
        targets     = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict   = model(images, targets)
        loss        = sum(loss for loss in loss_dict.values())
        loss_total += loss
        loss.backward()
        optimizer.step()

    return loss_total / len(train_loader)


In [14]:
!pip install termcolor2
from termcolor2 import colored 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for termcolor2: filename=termcolor2-0.0.3-py3-none-any.whl size=1833 sha256=e89cdcea134571e6c82c2e541530ed16c6c9315d671811c18ea9b2d5a820fc90
  Stored in directory: /root/.cache/pip/wheels/37/93/fa/3ead517c143d4381f9b4096524f21b1d86a5fa60d08210abe8
Successfully built termcolor2


In [26]:
import datetime
train_epochs = 100

for epoch in range(train_epochs):
    s_time = datetime.datetime.now()
    loss   = train_one_epoch(light_model, optimizer, trainloader)
    e_time = datetime.datetime.now()
    train_time = str(e_time - s_time)
    print(colored("Epoch: [{}] \t ".format(epoch + 1)),
          colored(f"Time: {train_time} \t ", color = "blue"),
          colored("Loss: {0:.4} \t ".format(loss), color = "red"),
          colored("LR: {}  ".format(lr_scheduler.get_last_lr()), color = "green")) 


    lr_scheduler.step()

Epoch: [1] 	  Time: 0:00:32.612377 	  Loss: 0.4779 	  LR: [0.0002]  
Epoch: [2] 	  Time: 0:00:32.714098 	  Loss: 0.3307 	  LR: [0.0002]  
Epoch: [3] 	  Time: 0:00:32.722975 	  Loss: 0.2577 	  LR: [0.0002]  
Epoch: [4] 	  Time: 0:00:33.765360 	  Loss: 0.2303 	  LR: [0.0002]  
Epoch: [5] 	  Time: 0:00:34.092729 	  Loss: 0.222 	  LR: [0.0002]  
Epoch: [6] 	  Time: 0:00:34.298914 	  Loss: 0.1834 	  LR: [0.0002]  
Epoch: [7] 	  Time: 0:00:34.250955 	  Loss: 0.166 	  LR: [0.0002]  
Epoch: [8] 	  Time: 0:00:34.574542 	  Loss: 0.1543 	  LR: [0.0002]  
Epoch: [9] 	  Time: 0:00:34.333593 	  Loss: 0.1808 	  LR: [0.0002]  
Epoch: [10] 	  Time: 0:00:34.306782 	  Loss: 0.1482 	  LR: [0.0002]  
Epoch: [11] 	  Time: 0:00:34.509424 	  Loss: 0.1221 	  LR: [0.0001]  
Epoch: [12] 	  Time: 0:00:34.337437 	  Loss: 0.1093 	  LR: [0.0001]  
Epoch: [13] 	  Time: 0:00:34.943607 	  Loss: 0.09688 	  LR: [0.0001]  
Epoch: [14] 	  Time: 0:00:34.580905 	  Loss: 0.09904 	  LR: [0.0001]  
Epoch: [15] 	  Time: 0:00:34.

In [27]:
!mkdir /content/MobileNet

In [31]:
light_model.eval()
with torch.no_grad():
    cnt = 0
    for images, targets in testloader:
        images  = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        out     = light_model(images)

In [32]:
out

[{'boxes': tensor([[ 23.4636,   2.3790, 474.8974, 484.1474]], device='cuda:0'),
  'labels': tensor([1], device='cuda:0'),
  'scores': tensor([0.9997], device='cuda:0')}]

In [33]:
from matplotlib.patches import Rectangle


def evaluate(model, test_loader):
    model.eval()
    with torch.no_grad():
        cnt = 0
        for images, targets in test_loader:
            images  = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            out     = model(images)
            scores  = out[0]["scores"].cpu().numpy()
            inds    = scores > 0.7
            boxes   = out[0]["boxes"].cpu().numpy()
            bxs     = boxes[inds]

            # ground Truth & image
            gt      = targets[0]["boxes"].cpu().numpy()
            image   = images[0].permute(1, 2, 0).cpu().numpy()
            #---------------------------------------------
            fig, ax = plt.subplots(1)
            ax.imshow(image)
            for j in range(len(gt)):
                rect1 = Rectangle((int(gt[j][0]), int(gt[j][1])), 
                                   abs(gt[j][2] - gt[j][0]),
                                   abs(gt[j][3] - gt[j][1]),
                                   fill = False, edgecolor = (0, 1, 0), linewidth = 3)
                ax.add_patch(rect1)

            for i in range(len(bxs)):
                rect2 = Rectangle((int(bxs[i][0]), int(bxs[i][1])),
                                   abs(bxs[i][2] - bxs[i][0]),
                                   abs(bxs[i][3] - bxs[i][1]),
                                   fill = False, edgecolor = (1, 0, 0), linewidth = 3)
                ax.add_patch(rect2)

            fig.savefig("/content/MobileNet/{}.png".format(cnt), dpi = 90, bbox_inches = "tight")
            cnt += 1




In [ ]:
evaluate(light_model, testloader)

## change the anchorgenerator class

In [ ]:
backbone = torchvision.models.mobilenet_v2(pretrained = True).features
backbone.out_channels = 1280

anchor_gen = torchvision.models.detection.rpn.AnchorGenerator(sizes = ((32, 64, 128, 256, 512), ),
                                                      aspect_ratios = ((0.5, 1.0, 2.0), ))

light_model2 = torchvision.models.detection.FasterRCNN(backbone = backbone, num_classes = 2,
                                                       rpn_anchor_generator = anchor_gen)

light_model2.to(device)
light_model2.train()
light_model2

In [42]:
x = torch.tensor([1, 2, 3, 4])
print(x.size())
y = torch.unsqueeze(x, 0)
print(y.size())
z = torch.unsqueeze(x, 1)
print(z.size())

torch.Size([4])
torch.Size([1, 4])
torch.Size([4, 1])


In [52]:
!git push https://github.com/HoseinEsk1994/FasterRCNN_resnet50_mobilenetv2.git

fatal: not a git repository (or any of the parent directories): .git
